<a href="https://colab.research.google.com/github/dropcreations/FFmpeg-for-Google-Colab/blob/main/FFmpeg_in_Google_Drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# __Install FFmpeg__

In [ ]:
#@markdown <br><center><img src='https://raw.githubusercontent.com/dropcreations/FFmpeg-for-Google-Colab/main/ffmpeg-logo.png' height="50" alt="FFmpeg-logo"/></center>
#@markdown <center><h3><b>Install FFmpeg 4.4</b></h3></center><br>
!sudo curl -L https://github.com/BtbN/FFmpeg-Builds/releases/download/latest/ffmpeg-n4.4-latest-linux64-gpl-4.4.tar.xz -o /usr/local/bin/ffmpeg.tar.xz
!sudo apt update
!sudo apt install p7zip-full p7zip-rar unrar rar
%cd /usr/local/bin/
!7z e /usr/local/bin/ffmpeg.tar.xz
!7z e /usr/local/bin/ffmpeg.tar
!sudo chmod a+rx /usr/local/bin/ffmpeg
%cd /content/
print("Successfully Installed.")

# __Mount Google Drive__

In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3><b>Mount Google Drive</b></h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

# __Run FFmpeg Scripts__

### __Remux Video files__

* **Remux** video files without re-encoding.
* Add file path and select a **output format**.
* Make sure that you **selected** output format **supports** to the all track types of input file.

In [ ]:
import os, sys, re
File_Path = "" #@param {type:"string"}
Output_Format = "mp4" #@param ["mp4", "mkv", "mov", "m4v"]

delsplit = re.search("\/(?:.(?!\/))+$", File_Path)
output_file_path = re.search("^[\/].+\/", File_Path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)

os.environ['inputFile'] = File_Path
os.environ['outputPath'] = output_file_path.group(0)
os.environ['fileName'] = filename_raw
os.environ['fileType'] = Output_Format

!ffmpeg -hide_banner -i "$inputFile" -map 0 -c copy -strict experimental "$outputPath"/"$fileName"-REMUX."$fileType"

### __Trim Video Files__

* **Trim** video files without re-encoding.
* Add file path and time duration to trim.

In [ ]:
import os, sys, re

File_Path = "" #@param {type:"string"}
Start_Time = "00:00:00.000" #@param {type:"string"}
End_Time = "00:00:00.000" #@param {type:"string"}

output_file_path = re.search("^[\/].+\/", File_Path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", File_Path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = File_Path
os.environ['outputPath'] = output_file_path_raw
os.environ['startTime'] = Start_Time
os.environ['endTime'] = End_Time
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!ffmpeg -hide_banner -i "$inputFile" -map 0 -ss "$startTime" -to "$endTime" -c copy "$outputPath"/"$fileName"-TRIM."$fileExtension"

### __Extract Audio__

* **Extract** audio from video files.
* When extracting DTS and TrueHD tracks, select **.mka**

In [ ]:
import os, sys, re

File_Path = "" #@param {type:"string"}
Output_Format = "m4a" #@param ["m4a", "mka", "eac3", "ac3", "aac", "mp3", "opus", "flac", "wav"]

delsplit = re.search("\/(?:.(?!\/))+$", File_Path)
output_file_path = re.search("^[\/].+\/", File_Path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)

os.environ['inputFile'] = File_Path
os.environ['outputPath'] = output_file_path.group(0)
os.environ['fileName'] = filename_raw
os.environ['fileType'] = Output_Format

!ffmpeg -hide_banner -i "$inputFile" -vn -c:a copy "$outputPath"/"$fileName"-AUDIO."$fileType"

### __Remove Bitstream Metadata__

* For a **AVC (H.264)** track run below cell.
* Output file will be saved to the **input file's folder**.

In [ ]:
import os, sys, re

Input_file = "" #@param {type:"string"}

cmd = output_file_path = re.search("^[\/].+\/", Input_file)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", Input_file)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = Input_file
os.environ['outputPath'] = output_file_path_raw
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!ffmpeg -hide_banner -i "$inputFile" -map 0 -c copy -bitexact -map_metadata -1 -vbsf filter_units=remove_types=6 "$outputPath"/"$fileName"-FIXED."$fileExtension"
clear_output()
print("Complete")

* For a **HEVC (H.265)** track run below cell.
* Output is a **raw bitstream** file. (**.hevc**)

In [ ]:
Input_file = "" #@param {type:"string"}
Output_file = "" #@param {type:"string"}

!ffmpeg -hide_banner -i "{Input_file}" -c:v copy -vbsf hevc_mp4toannexb source-video.hevc
!ffmpeg -hide_banner -i source-video.hevc -c copy -vbsf filter_units=remove_types=39 "{Output_file}.hevc"
clear_output()
print("Complete")

### __Encode H.264 | CRF__

* Enter file paths and output format.
* **Change** video settings as you want.
* If you also want to **encode the audio** track, select a **codec**.

* `aac`        - Advanced Audio Codec (AAC) <br>
* `eac3`       - Dolby Digital Plus (EAC-3) <br>
* `pcm_s24le`  - Lossless WAV (24bit) <br>
* `flac`       - Lossless FLAC <br>
* `libopus`    - Opus Audio <br>

* Change **bitrate** and **channel** settings to get various audio **qualities**.

* `2` - Stereo
* `6` - Surround 5.1
* `8` - Surround 7.1

In [ ]:
Input_file = ""  # @param {type:"string"}
Output_file = ""  # @param {type:"string"}
Output_format = "mkv" #@param ["mp4", "mkv", "mov"]
#@markdown <h3><b>Video Settings</b></h3>
Preset = "veryslow" #@param ["ultrafast", "superfast", "veryfast", "faster", "fast", "medium", "slow", "slower", "veryslow", "placebo"]
Profile = "high" #@param ["baseline", "main", "high", "high10", "high422", "high444"]
Level = "4" #@param ["1", "1.1", "1.2", "1.3", "2", "2.1", "2.2", "3", "3.1","3.2", "4", "4.1", "4.2", "5", "5.1", "5.2"]
Tune = "None" #@param ["None", "film", "animation", "grain", "stillimage", "psnr", "ssim", "fastdecode", "zerolatency"]
CRF = 19 #@param {type:"slider", min:0, max:51, step:1}
#@markdown <h3><b>Audio Settings</b></h3>
Audio_codec = "copy" #@param ["copy", "aac", "eac3", "pcm_s24le", "flac", "libopus"]
Audio_bitrate = "320"  # @param {type:"string"}
Audio_channels = "2" #@param ["2", "6", "8"]

if Tune == "None":
  cmd = !ffmpeg -i "{Input_file}" -c:v libx264 -crf $CRF -preset $Preset -profile:v $Profile -level $Level -maxrate 20000k -bufsize 25000k -partitions all -me_range 120 -refs 4 -keyint_min 121 -rc-lookahead 240 -nal-hrd 1 -sc_threshold 0 -threads 8 -x264-params "colorprim=bt709:colormatrix=bt709:transfer=bt709:qpmin=6:qpmax=51:bframes=16:lookahead-threads=2:stitchable=1:keyint=240" -bitexact -map_metadata -1 -vbsf filter_units=remove_types=6 -c:a $Audio_codec -b:a {Audio_bitrate}k -ac $Audio_channels "{Output_file}.{Output_format}"
  !$cmd
else:
  cmd = !ffmpeg -i "{Input_file}" -c:v libx264 -crf $CRF -tune {Tune} -preset $Preset -profile:v $Profile -level $Level -maxrate 20000k -bufsize 25000k -partitions all -me_range 120 -refs 4 -keyint_min 121 -rc-lookahead 240 -nal-hrd 1 -sc_threshold 0 -threads 8 -x264-params "colorprim=bt709:colormatrix=bt709:transfer=bt709:qpmin=6:qpmax=51:bframes=16:lookahead-threads=2:stitchable=1:keyint=240" -bitexact -map_metadata -1 -vbsf filter_units=remove_types=6 -c:a $Audio_codec -b:a {Audio_bitrate}k -ac $Audio_channels "{Output_file}.{Output_format}"
  !$cmd

### __Encode H.264 | 2-Pass__

* Enter file paths and output format.
* **Change** video settings as you want.
* If you also want to **encode the audio** track, select a **codec**.
* **X** = Bitrate **/ 2** ,<br>
  **Maxrate** = Bitrate + **X**<br>
  **Bufsize** = Maxrate + **X**

* `aac`        - Advanced Audio Codec (AAC) <br>
* `eac3`       - Dolby Digital Plus (EAC-3) <br>
* `pcm_s24le`  - Lossless WAV (24bit) <br>
* `flac`       - Lossless FLAC <br>
* `libopus`    - Opus Audio <br>

* Change **bitrate** and **channel** settings to get various audio **qualities**.

* `2` - Stereo
* `6` - Surround 5.1
* `8` - Surround 7.1

In [ ]:
Input_file = ""  #@param {type:"string"}
Output_file = ""  #@param {type:"string"}
Output_format = "mkv" #@param ["mp4", "mkv", "mov"]
#@markdown <h3><b>Video Settings</b></h3>
Preset = "veryslow" #@param ["ultrafast", "superfast", "veryfast", "faster", "fast", "medium", "slow", "slower", "veryslow", "placebo"]
Profile = "high" #@param ["baseline", "main", "high", "high10", "high422", "high444"]
Level = "4" #@param ["1", "1.1", "1.2", "1.3", "2", "2.1", "2.2", "3", "3.1","3.2", "4", "4.1", "4.2", "5", "5.1", "5.2"]
Tune = "None" #@param ["None", "film", "animation", "grain", "stillimage", "psnr", "ssim", "fastdecode", "zerolatency"]
Bitrate = "5800" #@param {type:"string"}
Maxrate = "8700" #@param {type:"string"}
Bufsize = "11600" #@param {type:"string"}
#@markdown <h3><b>Audio Settings</b></h3>
Audio_codec = "copy" #@param ["copy", "aac", "eac3", "pcm_s24le", "flac", "libopus"]
Audio_bitrate = "320"  #@param {type:"string"}
Audio_channels = "2" #@param ["2", "6", "8"]

if Tune == "None":
  cmd = !ffmpeg -i "{Input_file}" -c:v libx264 -pass 1 -b:v {Bitrate}k -preset $Preset -profile:v $Profile -level $Level -maxrate {Maxrate}k -bufsize {Bufsize}k -partitions all -me_range 24 -refs 4 -keyint_min 24 -rc-lookahead 48 -nal-hrd 1 -sc_threshold 0 -threads 8 -x264-params "colorprim=bt709:colormatrix=bt709:transfer=bt709:qpmin=6:qpmax=51:bframes=4:stitchable=1:keyint=48" -an -y -f null /dev/null
  !$cmd
  cmd = !ffmpeg -i "{Input_file}" -c:v libx264 -pass 2 -b:v {Bitrate}k -preset $Preset -profile:v $Profile -level $Level -maxrate {Maxrate}k -bufsize {Bufsize}k -partitions all -me_range 24 -refs 4 -keyint_min 24 -rc-lookahead 48 -nal-hrd 1 -sc_threshold 0 -threads 8 -x264-params "colorprim=bt709:colormatrix=bt709:transfer=bt709:qpmin=6:qpmax=51:bframes=4:stitchable=1:keyint=48" -bitexact -map_metadata -1 -vbsf filter_units=remove_types=6 -c:a $Audio_codec -b:a {Audio_bitrate}k -ac $Audio_channels "{Output_file}.{Output_format}"
  !$cmd
else:
  cmd = !ffmpeg -i "{Input_file}" -c:v libx264 -pass 1 -b:v {Bitrate}k -tune {Tune} -preset $Preset -profile:v $Profile -level $Level -maxrate {Maxrate}k -bufsize {Bufsize}k -partitions all -me_range 24 -refs 4 -keyint_min 24 -rc-lookahead 48 -nal-hrd 1 -sc_threshold 0 -threads 8 -x264-params "colorprim=bt709:colormatrix=bt709:transfer=bt709:qpmin=6:qpmax=51:bframes=4:stitchable=1:keyint=48" -an -y -f null /dev/null
  !$cmd
  cmd = !ffmpeg -i "{Input_file}" -c:v libx264 -pass 2 -b:v {Bitrate}k -tune {Tune} -preset $Preset -profile:v $Profile -level $Level -maxrate {Maxrate}k -bufsize {Bufsize}k -partitions all -me_range 24 -refs 4 -keyint_min 24 -rc-lookahead 48 -nal-hrd 1 -sc_threshold 0 -threads 8 -x264-params "colorprim=bt709:colormatrix=bt709:transfer=bt709:qpmin=6:qpmax=51:bframes=4:stitchable=1:keyint=48" -bitexact -map_metadata -1 -vbsf filter_units=remove_types=6 -c:a $Audio_codec -b:a {Audio_bitrate}k -ac $Audio_channels "{Output_file}.{Output_format}"
  !$cmd